In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch as t
from torch.utils import data
from torchvision import transforms as tsf

In [44]:
import torch.nn as nn
import torchvision.models as models


class FCN_32s(nn.Module):
    
    
    def __init__(self, num_classes=1000):
        
        super(FCN_32s, self).__init__()
        
        # Load the model with convolutionalized
        # fully connected layers
        vgg16 = models.vgg16(pretrained=False)
        
        # Copy all the feature layers as is
        self.features = vgg16.features
              
        '''# Remove the last classification 1x1 convolution
        # because it comes from imagenet 1000 class classification.
        # We will perform classification on different classes
        fully_conv = list(vgg16.classifier.children())
        fully_conv = fully_conv[:-1]
        self.fully_conv = nn.Sequential(*fully_conv)
        
        # Get a new 1x1 convolution and randomly initialize
        score_32s = nn.Linear(4096, num_classes, 1)
        self._normal_initialization(score_32s)
        self.score_32s = score_32s
        '''
        self.fc = nn.Sequential(
            nn.Linear(25088,4096),            
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096,4096),            
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(in_features=4096, out_features=1)
        )
        
    def _normal_initialization(self, layer):
        
        layer.weight.data.normal_(0, 0.01)
        layer.bias.data.zero_()
        
    def forward(self, x):
        
        #input_spatial_dim = x.size()[2:]
        
        x = self.features(x)
        #x = self.fully_conv(x)
        #x=self.fc(x)
        #x = self.score_32s(x)
        x = x.view(x.size(0), -1)
        x=self.fc(x)
        #x = nn.functional.upsample_bilinear(input=x, size=input_spatial_dim)
        
        return x

In [45]:
model = FCN_32s(num_classes=1)
images = t.autograd.Variable(t.randn(1, 3,224,224)) # => randn(10, 3, 224, 224)
outputs = model(images)

ValueError: Expected 4D tensor as input, got 2D tensor instead.

In [13]:
import torchvision.models as models

In [21]:
vgg16 = models.vgg16(pretrained=True)
images = t.autograd.Variable(t.randn(1, 3,128,128)) # => randn(10, 3, 224, 224)
outputs = vgg16(images)

RuntimeError: size mismatch, m1: [1 x 8192], m2: [25088 x 4096] at /opt/conda/conda-bld/pytorch_1513368888240/work/torch/lib/TH/generic/THTensorMath.c:1416

In [25]:
print(model)

FCN_32s(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size=(3, 

In [26]:
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size=(3, 3), 